In [2]:
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pandas as pd
from keras.models import load_model
import numpy as np
from sklearn.cluster import KMeans
import copy 

In [66]:
data = pd.read_csv('data_hackathon_v04.csv',sep='|')
data.head()

In [67]:
X = np.array(data['coordinate_from'].tolist())
k_m_dens = KMeans(n_clusters = 100, random_state=1).fit(X)
data['labels_by_density'] = k_m_dens.predict(X)
data['new_coordinate_by_dens'] = k_m_dens.labels_

In [68]:
del data['new_coordinate_by_dens']
test1 = pd.DataFrame(index = [i for i in range(100)])
test1['labels_by_density'] = [i for i in range(100)]
new_coordinate_lat = [i[0] for i in k_m_dens.cluster_centers_ ]
new_coordinate_lon = [i[1] for i in k_m_dens.cluster_centers_ ]
test1['new_coordinate_lat'] = new_coordinate_lat
test1['new_coordinate_lon'] = new_coordinate_lon
test1['new_coordinate_by_dens'] = test1.apply(lambda x: [x['new_coordinate_lat']] + [x['new_coordinate_lon']] , axis=1)
del test1['new_coordinate_lat'], test1['new_coordinate_lon']
# test1.head()

In [69]:
df = pd.merge(data,test1, how='left', on='labels_by_density')


In [11]:
# df = pd.read_csv("max.csv")
df['per_hour'] = df['created_at'].map(lambda x: pd.to_datetime(str(x)[:13]))
def fill_missed_hours()
    start = clusters[0].index.min()
    end = clusters[0].index.max()
    clusters = df.groupby(['labels_by_density', 'per_hour'])['ride_id'].count()
    new_cluster = []
    for cluster in clusters.index.get_level_values(0).unique():
        new = pd.Series(index = pd.date_range(start,end, freq='H'), data = 0)
        new[new.index.isin(clusters[cluster].index)] = clusters[cluster]
        new_cluster.append(new)
    return new_cluster
# new_cluster = fill_missed_hours

[2018-08-21 00:00:00     3
 2018-08-21 01:00:00     0
 2018-08-21 02:00:00     0
 2018-08-21 03:00:00     0
 2018-08-21 04:00:00     0
 2018-08-21 05:00:00     4
 2018-08-21 06:00:00     1
 2018-08-21 07:00:00     1
 2018-08-21 08:00:00     4
 2018-08-21 09:00:00    11
 2018-08-21 10:00:00    10
 2018-08-21 11:00:00     4
 2018-08-21 12:00:00     1
 2018-08-21 13:00:00     2
 2018-08-21 14:00:00     3
 2018-08-21 15:00:00     2
 2018-08-21 16:00:00     2
 2018-08-21 17:00:00     2
 2018-08-21 18:00:00     1
 2018-08-21 19:00:00     5
 2018-08-21 20:00:00     3
 2018-08-21 21:00:00     3
 2018-08-21 22:00:00     4
 2018-08-21 23:00:00     1
 2018-08-22 00:00:00     0
 2018-08-22 01:00:00     0
 2018-08-22 02:00:00     0
 2018-08-22 03:00:00     0
 2018-08-22 04:00:00     1
 2018-08-22 05:00:00     4
                        ..
 2019-02-20 10:00:00     7
 2019-02-20 11:00:00     6
 2019-02-20 12:00:00     5
 2019-02-20 13:00:00     5
 2019-02-20 14:00:00     2
 2019-02-20 15:00:00     2
 

In [23]:
def create_dataset(dataset, window_size = 169):
    data_X, data_Y = [], []
    for i in range(len(dataset) - window_size - 1):
        a = dataset[i:(i + window_size), 0]
        data_X.append(a)
        data_Y.append(dataset[i + window_size, 0])
    return(np.array(data_X), np.array(data_Y))

In [26]:
def predict_and_score(model, X, Y):
    # Make predictions on the original scale of the data.
    pred = scaler.inverse_transform(model.predict(X))
    # Prepare Y data to also be on the original scale for interpretability.
    orig_data = scaler.inverse_transform([Y])
    # Calculate RMSE.
    print(len(orig_data[0]))
    score = math.sqrt(mean_squared_error(orig_data[0], pred[:, 0]))
    return(score, pred)

In [27]:
def fit_model(train_X, train_Y, window_size = 169):
    model = Sequential()
    
    model.add(LSTM(6, 
                   input_shape = (1, window_size)))
    model.add(Dense(1))
    model.compile(loss = "mean_squared_error", 
                  optimizer = "adam")
    model.fit(train_X, 
              train_Y, 
              epochs = 20, 
              batch_size = 1, 
              verbose = 2)
    
    return(model)

In [57]:
# rmse = []

for i in range(80,90):
    my_data = pd.DataFrame(new_cluster[i]).reset_index()
    my_data.columns = ['date','freq']
    data_raw = my_data['freq'].values.astype('float32').reshape(-1,1)
    scaler = MinMaxScaler(feature_range = (0, 1))
    dataset = scaler.fit_transform(data_raw)
    
    # Create test and training sets for one-step-ahead regression.
    window_size = 169
    test = dataset[-338:]
    train = dataset[:-338]
    train_X, train_Y = create_dataset(train, window_size)
    test_X, test_Y = create_dataset(test, window_size)
    print("Original training data shape:")
    print(train_X.shape)

    # Reshape the input data into appropriate form for Keras.
    train_X = np.reshape(train_X, (train_X.shape[0], 1, train_X.shape[1]))
    test_X = np.reshape(test_X, (test_X.shape[0], 1, test_X.shape[1]))
    print('model :'+str(i))
    print("New training data shape "+str(i+1)+':')
    print(train_X.shape)
    print('__________')
    
    model1 = fit_model(train_X, train_Y, window_size)
    
    model1.save('my_model_'+str(i+1)+'.h5')  # creates a HDF5 file 'my_model...h5'
    
    rmse_train, train_predict = predict_and_score(model1, train_X, train_Y)
    rmse_test, test_predict = predict_and_score(model1, test_X, test_Y)
    
    rmse.append(rmse_test)
    
    print("Training data score: %.2f RMSE" % rmse_train)
    print("Test data score: %.2f RMSE" % rmse_test)


Original training data shape:
(3924, 169)
model :80
New training data shape 81:
(3924, 1, 169)
__________
Epoch 1/20
 - 17s - loss: 0.0013
Epoch 2/20
 - 9s - loss: 0.0012
Epoch 3/20
 - 9s - loss: 0.0012
Epoch 4/20
 - 9s - loss: 0.0011
Epoch 5/20
 - 9s - loss: 0.0011
Epoch 6/20
 - 10s - loss: 0.0011
Epoch 7/20
 - 11s - loss: 0.0011
Epoch 8/20
 - 11s - loss: 0.0010
Epoch 9/20
 - 9s - loss: 0.0010
Epoch 10/20
 - 11s - loss: 0.0010
Epoch 11/20
 - 10s - loss: 9.9066e-04
Epoch 12/20
 - 12s - loss: 9.5794e-04
Epoch 13/20
 - 9s - loss: 9.4959e-04
Epoch 14/20
 - 11s - loss: 9.2326e-04
Epoch 15/20
 - 9s - loss: 9.0620e-04
Epoch 16/20
 - 11s - loss: 8.7423e-04
Epoch 17/20
 - 10s - loss: 8.5906e-04
Epoch 18/20
 - 12s - loss: 8.3945e-04
Epoch 19/20
 - 9s - loss: 8.3258e-04
Epoch 20/20
 - 9s - loss: 8.0422e-04
3924
168
Training data score: 0.41 RMSE
Test data score: 0.34 RMSE
Original training data shape:
(3924, 169)
model :81
New training data shape 82:
(3924, 1, 169)
__________
Epoch 1/20
 - 19s -

Epoch 16/20
 - 12s - loss: 0.0016
Epoch 17/20
 - 11s - loss: 0.0016
Epoch 18/20
 - 13s - loss: 0.0015
Epoch 19/20
 - 12s - loss: 0.0015
Epoch 20/20
 - 13s - loss: 0.0015
3924
168
Training data score: 0.33 RMSE
Test data score: 0.31 RMSE


In [58]:
r = rmse.copy()

In [59]:
import copy 

In [60]:
np.max(rmse), np.min(rmse)

(5.951919175112068, 6.221281364560127e-05)

In [61]:
np.max(rmse), np.min(rmse)

(5.951919175112068, 6.221281364560127e-05)

In [62]:
np.mean(rmse)

1.795466313894356

In [63]:
np.mean(rmse)

1.795466313894356